In [3]:
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module=r"louvain")

import pandas as pd
import numpy as np
import scanpy as sc

# paths
repo_root = Path("/Users/oliviamohning/Documents/ds-portfolio/scrnaseq")
data_dir  = repo_root / "data"
src_dir   = data_dir / "GSE138852"
proc_dir  = data_dir / "processed"
fig_dir   = repo_root / "figures"
res_dir   = repo_root / "results"

for d in (proc_dir, fig_dir, res_dir):
    d.mkdir(parents=True, exist_ok=True)

counts_path = src_dir / "GSE138852_counts.csv"
meta_path   = src_dir / "GSE138852_covariates.csv"

counts_path, meta_path

(PosixPath('/Users/oliviamohning/Documents/ds-portfolio/scrnaseq/data/GSE138852/GSE138852_counts.csv'),
 PosixPath('/Users/oliviamohning/Documents/ds-portfolio/scrnaseq/data/GSE138852/GSE138852_covariates.csv'))